# 4-3-4 ネットワークモデルの作成

- 本ファイルでは、OpenPoseのネットワークモデルと順伝搬forward関数を作成します。


# 4.3 学習目標

1.	OpenPoseのネットワーク構造をモジュール単位で理解する
2.	OpenPoseを構成する各モジュールのサブネットワーク構成を理解する
3.	OpenPoseのネットワーククラスを実装できるようになる



# 4.4 学習目標

1.	Featureモジュールのサブネットワーク構成を理解する
2.	Featureモジュールを実装できるようになる
3.	Stageモジュールのblockを構成するサブネットワークを理解する
4.	Stageモジュールを実装できるようになる


# 事前準備


とくになし

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd "/content/drive/My Drive/Colab Notebooks/pytorch_advanced/4_pose_estimation"

/content/drive/My Drive/Colab Notebooks/pytorch_advanced/4_pose_estimation


In [3]:
# 必要なパッケージのimport
import torch
import torch.nn as nn
from torch.nn import init
import torchvision

In [4]:
class OpenPoseNet(nn.Module):
    def __init__(self):
        super(OpenPoseNet, self).__init__()

        # Featureモジュール
        self.model0 = OpenPose_Feature()

        # Stageモジュール
        # PAFs（Part Affinity Fields）側
        self.model1_1 = make_OpenPose_block('block1_1')
        self.model2_1 = make_OpenPose_block('block2_1')
        self.model3_1 = make_OpenPose_block('block3_1')
        self.model4_1 = make_OpenPose_block('block4_1')
        self.model5_1 = make_OpenPose_block('block5_1')
        self.model6_1 = make_OpenPose_block('block6_1')

        # confidence heatmap側
        self.model1_2 = make_OpenPose_block('block1_2')
        self.model2_2 = make_OpenPose_block('block2_2')
        self.model3_2 = make_OpenPose_block('block3_2')
        self.model4_2 = make_OpenPose_block('block4_2')
        self.model5_2 = make_OpenPose_block('block5_2')
        self.model6_2 = make_OpenPose_block('block6_2')

    def forward(self, x):
        """順伝搬の定義"""

        # Featureモジュール
        out1 = self.model0(x)

        # Stage1
        out1_1 = self.model1_1(out1)  # PAFs側
        out1_2 = self.model1_2(out1)  # confidence heatmap側

        # CStage2
        out2 = torch.cat([out1_1, out1_2, out1], 1)  # 次元1のチャネルで結合
        out2_1 = self.model2_1(out2)
        out2_2 = self.model2_2(out2)

        # Stage3
        out3 = torch.cat([out2_1, out2_2, out1], 1)
        out3_1 = self.model3_1(out3)
        out3_2 = self.model3_2(out3)

        # Stage4
        out4 = torch.cat([out3_1, out3_2, out1], 1)
        out4_1 = self.model4_1(out4)
        out4_2 = self.model4_2(out4)

        # Stage5
        out5 = torch.cat([out4_1, out4_2, out1], 1)
        out5_1 = self.model5_1(out5)
        out5_2 = self.model5_2(out5)

        # Stage6
        out6 = torch.cat([out5_1, out5_2, out1], 1)
        out6_1 = self.model6_1(out6)
        out6_2 = self.model6_2(out6)

        # 損失の計算用に各Stageの結果を格納
        saved_for_loss = []
        saved_for_loss.append(out1_1)  # PAFs側
        saved_for_loss.append(out1_2)  # confidence heatmap側
        saved_for_loss.append(out2_1)
        saved_for_loss.append(out2_2)
        saved_for_loss.append(out3_1)
        saved_for_loss.append(out3_2)
        saved_for_loss.append(out4_1)
        saved_for_loss.append(out4_2)
        saved_for_loss.append(out5_1)
        saved_for_loss.append(out5_2)
        saved_for_loss.append(out6_1)
        saved_for_loss.append(out6_2)

        # 最終的なPAFsのout6_1とconfidence heatmapのout6_2、そして
        # 損失計算用に各ステージでのPAFsとheatmapを格納したsaved_for_lossを出力
        # out6_1：torch.Size([minibatch, 38, 46, 46])
        # out6_2：torch.Size([minibatch, 19, 46, 46])
        # saved_for_loss:[out1_1, out_1_2, ・・・, out6_2]

        return (out6_1, out6_2), saved_for_loss


# Featureモジュール

In [5]:
class OpenPose_Feature(nn.Module):
    def __init__(self):
        super(OpenPose_Feature, self).__init__()

        # VGG-19の最初10個の畳み込みを使用
        # 初めて実行する際は、モデルの重みパラメータをダウンロードするため、実行に時間がかかる
        vgg19 = torchvision.models.vgg19(pretrained=True)
        model = {}
        model['block0'] = vgg19.features[0:23]  # VGG19の最初の10個の畳み込み層まで

        # 残りは新たな畳み込み層を2つ用意
        model['block0'].add_module("23", torch.nn.Conv2d(
            512, 256, kernel_size=3, stride=1, padding=1))
        model['block0'].add_module("24", torch.nn.ReLU(inplace=True))
        model['block0'].add_module("25", torch.nn.Conv2d(
            256, 128, kernel_size=3, stride=1, padding=1))
        model['block0'].add_module("26", torch.nn.ReLU(inplace=True))

        self.model = model['block0']

    def forward(self, x):
        outputs = self.model(x)
        return outputs


# Stageモジュール

In [6]:
def make_OpenPose_block(block_name):
    """
    コンフィグレーション変数からOpenPoseのStageモジュールのblcokを作成
    nn.Moduleではなく、nn.Sequentialにする
    """

    # 1. コンフィグレーションの辞書変数blocksを作成し、ネットワークを生成させる
    # 最初に全パターンの辞書を用意し、引数block_nameのみを生成する
    blocks = {}
    # Stage 1
    blocks['block1_1'] = [{'conv5_1_CPM_L1': [128, 128, 3, 1, 1]},
                          {'conv5_2_CPM_L1': [128, 128, 3, 1, 1]},
                          {'conv5_3_CPM_L1': [128, 128, 3, 1, 1]},
                          {'conv5_4_CPM_L1': [128, 512, 1, 1, 0]},
                          {'conv5_5_CPM_L1': [512, 38, 1, 1, 0]}]

    blocks['block1_2'] = [{'conv5_1_CPM_L2': [128, 128, 3, 1, 1]},
                          {'conv5_2_CPM_L2': [128, 128, 3, 1, 1]},
                          {'conv5_3_CPM_L2': [128, 128, 3, 1, 1]},
                          {'conv5_4_CPM_L2': [128, 512, 1, 1, 0]},
                          {'conv5_5_CPM_L2': [512, 19, 1, 1, 0]}]

    # Stages 2 - 6
    for i in range(2, 7):
        blocks['block%d_1' % i] = [
            {'Mconv1_stage%d_L1' % i: [185, 128, 7, 1, 3]},
            {'Mconv2_stage%d_L1' % i: [128, 128, 7, 1, 3]},
            {'Mconv3_stage%d_L1' % i: [128, 128, 7, 1, 3]},
            {'Mconv4_stage%d_L1' % i: [128, 128, 7, 1, 3]},
            {'Mconv5_stage%d_L1' % i: [128, 128, 7, 1, 3]},
            {'Mconv6_stage%d_L1' % i: [128, 128, 1, 1, 0]},
            {'Mconv7_stage%d_L1' % i: [128, 38, 1, 1, 0]}
        ]

        blocks['block%d_2' % i] = [
            {'Mconv1_stage%d_L2' % i: [185, 128, 7, 1, 3]},
            {'Mconv2_stage%d_L2' % i: [128, 128, 7, 1, 3]},
            {'Mconv3_stage%d_L2' % i: [128, 128, 7, 1, 3]},
            {'Mconv4_stage%d_L2' % i: [128, 128, 7, 1, 3]},
            {'Mconv5_stage%d_L2' % i: [128, 128, 7, 1, 3]},
            {'Mconv6_stage%d_L2' % i: [128, 128, 1, 1, 0]},
            {'Mconv7_stage%d_L2' % i: [128, 19, 1, 1, 0]}
        ]

    # 引数block_nameのコンフィグレーション辞書を取り出す
    cfg_dict = blocks[block_name]

    # 2. コンフィグレーション内容をリスト変数layersに格納
    layers = []

    # 0番目から最後の層までを作成
    for i in range(len(cfg_dict)):
        for k, v in cfg_dict[i].items():
            if 'pool' in k:
                layers += [nn.MaxPool2d(kernel_size=v[0], stride=v[1],
                                        padding=v[2])]
            else:
                conv2d = nn.Conv2d(in_channels=v[0], out_channels=v[1],
                                   kernel_size=v[2], stride=v[3],
                                   padding=v[4])
                layers += [conv2d, nn.ReLU(inplace=True)]

    # 3. layersをSequentialにする
    # ただし、最後にReLUはいらないのでその手前までを使用する
    net = nn.Sequential(*layers[:-1])

    # 4. 初期化関数の設定し、畳み込み層を初期化する

    def _initialize_weights_norm(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.normal_(m.weight, std=0.01)
                if m.bias is not None:
                    init.constant_(m.bias, 0.0)

    net.apply(_initialize_weights_norm)

    return net


# 動作確認

In [7]:
# モデルの定義
net = OpenPoseNet()
net.train()

# ダミーデータの作成
batch_size = 2
dummy_img = torch.rand(batch_size, 3, 368, 368)

# 計算
outputs = net(dummy_img)
print(outputs)


Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth



((tensor([[[[-3.5642e-05, -2.1006e-05, -2.8925e-06,  ..., -4.4207e-05,
           -2.9037e-05, -1.0460e-05],
          [-5.7738e-05, -5.0310e-05, -3.5061e-05,  ..., -6.6725e-05,
           -3.8282e-05, -2.5124e-05],
          [-4.6770e-05, -3.9471e-05, -4.6771e-05,  ..., -5.3679e-05,
           -5.0885e-05, -2.4260e-05],
          ...,
          [ 6.3675e-06, -3.3793e-05, -3.9695e-05,  ..., -3.4262e-05,
           -3.7721e-05, -1.7271e-05],
          [-1.6225e-06, -3.8170e-05, -1.1770e-05,  ..., -2.1703e-05,
           -2.0703e-05, -1.2450e-05],
          [-4.5559e-06, -3.9396e-05, -3.3522e-05,  ..., -2.1669e-05,
           -8.7291e-06, -3.4571e-05]],

         [[ 7.9986e-07,  2.3278e-05, -8.9410e-06,  ...,  4.2165e-06,
           -2.0088e-06,  1.2065e-05],
          [-4.0284e-05, -2.8488e-05, -5.1157e-05,  ...,  4.4611e-06,
            1.0086e-06, -5.0142e-06],
          [-1.8104e-05, -2.1184e-06, -8.8335e-06,  ...,  3.0615e-05,
            3.3084e-05,  3.6480e-05],
          ...,
  

以上